# Google American Sign Language

So Google is again here with another great competition **`Google - American Sign Language Fingerspelling Recognition`**

Lets first understand the competition in detail 

* **American Sign Language FingerSpelling** - $American$ $Sign$ $Language$ $FingerSpelling$ is a method used in sign language to spell out individual letters or words. It involves using specific handshapes and movements to represent each letter of the alphabet. $FingerSpelling$ is an important tool for communication and is often used to convey names, places, or unfamiliar words in $ASL$. It requires practice and skill to execute accurately and fluently, and NO!!, (middle finger is not included in this language)

<img src = "https://d.newsweek.com/en/full/1394686/asl-getty-images.jpg">

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/asl-fingerspelling/supplemental_metadata.csv
/kaggle/input/asl-fingerspelling/character_to_prediction_index.json
/kaggle/input/asl-fingerspelling/train.csv
/kaggle/input/asl-fingerspelling/supplemental_landmarks/371169664.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/369584223.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1682915129.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/775880548.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/2100073719.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1650637630.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/1471096258.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/86446671.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/897287709.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/333606065.parquet
/kaggle/input/asl-fingerspelling/supplemental_landmarks/2057261717.parquet
/kaggle/inpu

In [2]:
pip install --pre torcharrow -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=25be98283c480adcafd79a9b23afde8d253b7b7f6578df5e6d71fd7b95937ecf
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
  

In [3]:
import pandas as pd 
import torchdata
import torcharrow

# 1 | Train 

So this our `train data`, this data contains all the information related to the the `training` and `target` values. As of I think `file_id` , `sequence_id` , `participant_id`, are not that important for the model, and thus it would be better if we just remove them

In [4]:
train = pd.read_csv("/kaggle/input/asl-fingerspelling/train.csv")
train

,path,file_id,sequence_id,participant_id,phrase
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah
2,train_landmarks/5414471.parquet,5414471,1816862427,0,hentaihubs.com
3,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier
4,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane
...,...,...,...,...,...
67282,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913
67283,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850
67284,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com
67285,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle


The `path` shows where the `parquet file` is located. You could also have just used the `file_id` for lacating the corresponding `parquet_file`, as if we split the `path` into `train_landmarks/` and the other part, we basically get the same value. 

We used `path` instead of the `file_id`. As file paths are easier to process when we try to find the data

The other column `phrase` is the target column, we will train for 

# 2 | Supplemental MetData

Suplement data looks like the same, its just we will test our model on this data

In [5]:
supplemental_metdata = pd.read_csv("/kaggle/input/asl-fingerspelling/supplemental_metadata.csv")
supplemental_metdata

,path,file_id,sequence_id,participant_id,phrase
0,supplemental_landmarks/33432165.parquet,33432165,1535467051,251,coming up with killer sound bites
1,supplemental_landmarks/33432165.parquet,33432165,1535499058,239,we better investigate this
2,supplemental_landmarks/33432165.parquet,33432165,1535530550,245,interesting observation was made
3,supplemental_landmarks/33432165.parquet,33432165,1535545499,38,victims deserve more redress
4,supplemental_landmarks/33432165.parquet,33432165,1535585216,254,knee bone is connected to the thigh bone
...,...,...,...,...,...
52953,supplemental_landmarks/2100073719.parquet,2100073719,1090866442,239,want to join us for lunch
52954,supplemental_landmarks/2100073719.parquet,2100073719,1090966452,95,this phenomenon will never occur
52955,supplemental_landmarks/2100073719.parquet,2100073719,1091005846,40,the winner of the race
52956,supplemental_landmarks/2100073719.parquet,2100073719,1091011550,241,are you sure you want this


In [6]:
supplemental_metdata.drop(["file_id" , "sequence_id" , "participant_id"] , axis = 1 , inplace = True)
supplemental_metdata

,path,phrase
0,supplemental_landmarks/33432165.parquet,coming up with killer sound bites
1,supplemental_landmarks/33432165.parquet,we better investigate this
2,supplemental_landmarks/33432165.parquet,interesting observation was made
3,supplemental_landmarks/33432165.parquet,victims deserve more redress
4,supplemental_landmarks/33432165.parquet,knee bone is connected to the thigh bone
...,...,...
52953,supplemental_landmarks/2100073719.parquet,want to join us for lunch
52954,supplemental_landmarks/2100073719.parquet,this phenomenon will never occur
52955,supplemental_landmarks/2100073719.parquet,the winner of the race
52956,supplemental_landmarks/2100073719.parquet,are you sure you want this


# 3 | Parquet Files
Parquet files are a little bit complicated to work with. But you can find particular methods in `pandas` and `tensorflow`. 

`Pytorch` has a specialised library for **loading parquet data from pipelines/paths**. You can find it **[torch.datapipes.iter.ParquetDataFrameLoader](https://pytorch.org/data/main/generated/torchdata.datapipes.iter.ParquetDataFrameLoader.html)**, You need to install `torcharrow` to use this library, `torcharrow` is not in stable release, but you can still install it using 
```
! pip install --pre torcharrow -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
import torcharraow
```
Though even after installing it is showing error to me, I dont know if it is the problem with my system only or not. If you find any leads, please tell me :)

Here is the **[Github](https://github.com/pytorch/torcharrow)** for `torcharrow`

Here is the code for loading the data
```
torchdata.datapipes.iter.ParquetDataFrameLoader(train["path"])
```

Now lets see how we are given the data 

I dont know but the `sample_data` is not shown in the actual release of the notebook. but can be seen in the `edit` mode. 

It has $1,73,385$ rows and $1,630$ columns

In [7]:
! pip install --pre torcharrow -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
import torcharrow

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html


In [8]:
!pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.1.2 requires pandas!=1.4.0,<1.6,>1.1, but you have pandas 2.0.1 which is incompatible.
woodwork 0.23.0 requires pandas<2.0.0,>=1.4.3, but you have pandas 2.0.1 which is incompatible.
wfdb 4.1.0 requires pandas<2.0.0,>=1.0.0, but you have pandas 2.0.1 which is incompatible.
torcharrow 0.2.0a0.dev20230524 requires pandas<=1.3.5, but you have pandas 2.0.1 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.

In [9]:
sample_data = pd.read_parquet("/kaggle/input/asl-fingerspelling/supplemental_landmarks/1032110484.parquet")
sample_data

,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,...,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20
sequence_id,,,,,,,,,,,,,,,,,,,,,
1617884228,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617884228,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617884228,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617884228,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617884228,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658023233,203,0.553113,0.548173,0.547054,0.538057,0.548786,0.549475,0.551103,0.449462,0.552681,...,-0.386854,-0.421294,-0.192267,-0.336643,-0.367230,-0.357574,-0.197995,-0.323567,-0.348959,-0.343505
1658023233,204,0.550300,0.550015,0.548267,0.539283,0.550646,0.551023,0.551594,0.449115,0.552687,...,-0.507649,-0.547935,-0.267368,-0.415424,-0.426492,-0.409436,-0.263647,-0.384693,-0.400662,-0.393638
1658023233,205,0.555492,0.549407,0.547812,0.539245,0.550138,0.550807,0.552136,0.452131,0.553723,...,-0.493019,-0.543748,-0.243397,-0.395400,-0.411943,-0.396315,-0.246496,-0.370401,-0.384761,-0.374569


# 4 | Advisory

This data has both `video` as an `input` and `language` as `output`. One lead (as of I think) is we can use R-CNN(Reccurent-Convolution Neural Network). Video data works great on these types of models. 

Also we need to reduce the training time as much as possible. As of the competition says. 

**THAT IT FOR TODAY GUYS**

**WE WILL GO DEEPER INTO THE DATA IN THE UPCOMING VERSIONS**

**PLEASE COMMENT YOUR THOUGHTS, HIHGLY APPRICIATED**

**DONT FORGET TO MAKE AN UPVOTE, IF YOU LIKED MY WORK**

<img src = "https://i.imgflip.com/19aadg.jpg">

**PEACE OUT!!!**